This tutorial is automatically generated from [TestPyCellSortingTutorial.py](https://github.com/Chaste/Chaste/blob/develop/pychaste/test/tutorial/TestPyCellSortingTutorial.py) at revision [2cd31145369b](https://github.com/Chaste/Chaste/commit/2cd31145369b99ac6f9bd17ad1c916ec7de20b28).


## Introduction
This test is a demonstration of cell sorting using a Cellular Potts based framework.
It shows:
 * How to set up a Potts simulation
 * Working with labels
 
## The Test



In [ ]:
import chaste  # The PyChaste module
import chaste.cell_based  # Contains cell populations
import chaste.mesh  # Contains meshes
import chaste.visualization  # Visualization tools
import matplotlib.pyplot as plt  # Plotting
import numpy as np  # Matrix tools
# Set up MPI

### Test 1 - Cell sorting
The next test generates a collection of cells, there are two types of cells, labelled ones and non labelled ones,
there is differential adhesion between the cell types. For the parameters specified, the cells sort into separate types.



In [ ]:
chaste.cell_based.SetupNotebookTest() # Set up the test

First, we generate a `Potts` mesh. To create a `PottsMesh`, we can use the `PottsMeshGenerator`.
This generates a regular square-shaped mesh, in which all elements are the same size.
We have chosen an 8 by 8 block of elements each consisting of 4 by 4 ( = 16) lattice sites.



In [ ]:
generator = chaste.mesh.PottsMeshGenerator_2(50, 8, 4, 50, 8, 4)
mesh = generator.GetMesh()

Having created a mesh, we now create some cells. To do this, we the `CellsGenerator` helper class,
as before but this time the third argument is set to make all cells non-proliferative.



In [ ]:
differentiated_type = chaste.cell_based.DifferentiatedCellProliferativeType()
cell_generator = chaste.cell_based.CellsGenerator_UniformCellCycleModel_2()
cells = cell_generator.GenerateBasicRandom(
    mesh.GetNumElements(), differentiated_type
)

Before we make a CellPopulation we make a cell label and then assign this label to some randomly chosen cells.



In [ ]:
label = chaste.cell_based.CellLabel()
for eachCell in cells:
    if chaste.core.RandomNumberGenerator.Instance().ranf() < 0.5:
        eachCell.AddCellProperty(label)

Now we have a mesh and a set of cells to go with it, we can create a `CellPopulation`.



In [ ]:
cell_population = chaste.cell_based.PottsBasedCellPopulation_2(mesh, cells)

In order to visualize labelled cells we need to use the following command.



In [ ]:
cell_population.AddCellWriterCellLabelWriter()

PyChaste can do simple 3D rendering with VTK. We set up a VtkScene so that we can
see the population evovle in real time.



In [ ]:
scene = chaste.visualization.VtkScene_2()
scene.SetCellPopulation(cell_population)
scene.GetCellPopulationActorGenerator().SetShowPottsMeshEdges(True)
nb_manager = chaste.visualization.JupyterNotebookManager()
nb_manager.vtk_show(scene, height=600)

We then pass in the cell population into an `OffLatticeSimulation`, and set the output directory and end time



In [ ]:
simulator = chaste.cell_based.OnLatticeSimulation_2(cell_population)
simulator.SetOutputDirectory("Python/TestCellSorting")
simulator.SetEndTime(20.0)
simulator.SetSamplingTimestepMultiple(10)

We must now create one or more update rules, which determine the Hamiltonian in the Potts simulation.
For this test, we use two update rules based upon a volume constraint (`VolumeConstraintPottsUpdateRule`) and
differential adhesion between cells (`DifferentialAdhesionPottsUpdateRule`), set appropriate parameters, and
pass them to the `OnLatticeSimulation`.



In [ ]:
volume_constraint_update_rule = (
    chaste.cell_based.VolumeConstraintPottsUpdateRule_2()
)
volume_constraint_update_rule.SetMatureCellTargetVolume(16)
volume_constraint_update_rule.SetDeformationEnergyParameter(0.2)
simulator.AddUpdateRule(volume_constraint_update_rule)

We repeat the process for any other update rules.



In [ ]:
differential_adhesion_update_rule = (
    chaste.cell_based.DifferentialAdhesionPottsUpdateRule_2()
)
differential_adhesion_update_rule.SetLabelledCellLabelledCellAdhesionEnergyParameter(
    0.16
)
differential_adhesion_update_rule.SetLabelledCellCellAdhesionEnergyParameter(
    0.11
)
differential_adhesion_update_rule.SetCellCellAdhesionEnergyParameter(0.02)
differential_adhesion_update_rule.SetLabelledCellBoundaryAdhesionEnergyParameter(
    0.16
)
differential_adhesion_update_rule.SetCellBoundaryAdhesionEnergyParameter(0.16)
simulator.AddUpdateRule(differential_adhesion_update_rule)

Set up plotting



In [ ]:
scene_modifier = chaste.visualization.JupyterSceneModifier_2(nb_manager)
scene_modifier.SetVtkScene(scene)
scene_modifier.SetUpdateFrequency(1000)
simulator.AddSimulationModifier(scene_modifier)

To run the simulation, we call `Solve()`.



In [ ]:
scene.Start()
simulator.Solve()
chaste.cell_based.TearDownNotebookTest() # Tear down the test